## Códigos do experimento de Classificador automático de imagens

André Figueiredo de Almeida
RA: 164047

Igor Torrente
RA: 169820

In [ ]:
### IMPORT ##
import sklearn.linear_model as lm
import sklearn.neural_network as nn
from sklearn.decomposition import PCA
import pickle
import time

In [ ]:
### Função que extrai dados do banco de dados ###
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
### Pegando dados do banco de dados ###
def getData():
    train = {b'labels' : [], b'data' : []}

    for i in range(1, 6):
        aux = unpickle('cifar-10-python/cifar-10-batches-py/data_batch_' + str(i))
        train[b'labels'].extend(aux[b'labels'])
        train[b'data'].extend(aux[b'data'])

    train_x = train[b'data']    

    train_y = train[b'labels']

    test = unpickle('cifar-10-python/cifar-10-batches-py/test_batch')

    test_x = test[b'data']

    test_y = test[b'labels']
    
    return train_x, train_y, test_x, test_y

### III. Tratamento de dados

In [ ]:
def applyPCA(perc):
    train = {b'labels' : [], b'data' : []}

    for i in range(1, 6):
        aux = unpickle('cifar-10-python/cifar-10-batches-py/data_batch_' + str(i))
        train[b'labels'].extend(aux[b'labels'])
        train[b'data'].extend(aux[b'data'])

    train_x = train[b'data'][40000:]

    train_y = train[b'labels'][40000:]

    test_x = train[b'data'][:10000]

    test_y = train[b'labels'][:10000]

    # nao aplicar PCA
    if perc == -1:
        return train_x, train_y, test_x, test_y
    
    pca = PCA(n_components=perc)
    pca.fit(train_x)
    train_x_pca = pca.transform(train_x)
    test_x_pca = pca.transform(test_x)

    soma = 0
    for i in pca.explained_variance_ratio_:
        soma += i
    print("Dados removidos/reduzidos: ",soma)
    return train_x_pca, train_y, test_x_pca, test_y

Exemplo sem PCA

In [ ]:
train_x, train_y, test_x, test_y = applyPCA(-1)

tic = time.clock()

lr = lm.LogisticRegression(n_jobs = -1, solver="saga").fit(train_x, train_y)
print("Score: ", lr.score(test_x, test_y))

tac = time.clock()
print("Tempo: ", tac - tic, "s")

Exemplos com PCA

In [ ]:
# Com 85% de remoção

train_x_pca, train_y, test_x_pca, test_y = applyPCA(0.85)

tic = time.clock()

lr = lm.LogisticRegression(n_jobs = -1, solver="saga").fit(train_x_pca, train_y)
print("Score: ", lr.score(test_x_pca, test_y))

tac = time.clock()
print("Tempo: ", tac - tic, "s")

In [ ]:
# Com 93% de remoção

train_x_pca, train_y, test_x_pca, test_y = applyPCA(0.93)

tic = time.clock()

lr = lm.LogisticRegression(n_jobs = -1, solver="saga").fit(train_x_pca, train_y)
print("Score: ", lr.score(test_x_pca, test_y))

tac = time.clock()
print("Tempo: ", tac - tic, "s")

In [ ]:
# Com 95% de remoção

train_x_pca, train_y, test_x_pca, test_y = applyPCA(0.95)

tic = time.clock()

lr = lm.LogisticRegression(n_jobs = -1, solver="saga").fit(train_x_pca, train_y)
print("Score: ", lr.score(test_x_pca, test_y))

tac = time.clock()
print("Tempo: ", tac - tic, "s")

In [ ]:
# Com 97% de remoção

train_x_pca, train_y, test_x_pca, test_y = applyPCA(0.97)

tic = time.clock()

lr = lm.LogisticRegression(n_jobs = -1, solver="saga").fit(train_x_pca, train_y)
print("Score: ", lr.score(test_x_pca, test_y))

tac = time.clock()
print("Tempo: ", tac - tic, "s")

### IV. Soluções propostas

In [ ]:
### Preparando dados com e sem PCA ###

train_x, train_y, test_x, test_y = getData()

In [ ]:
pca = PCA(n_components=0.93)
pca.fit(train_x)
train_x_pca = pca.transform(train_x)
test_x_pca = pca.transform(test_x)

A. Regressão logística com um contra todos

1) Sem PCA

In [ ]:
lr = lm.LogisticRegression(n_jobs = -1, solver="saga").fit(train_x, train_y)
lr.score(test_x, test_y)

2) Com PCA

In [ ]:
lr = lm.LogisticRegression(n_jobs = -1, solver="saga").fit(train_x_pca, train_y)
lr.score(test_x_pca, test_y)

B. Regressão logística multinomial

1) Sem PCA

In [ ]:
lr = lm.LogisticRegression(n_jobs = -1, solver="saga", multi_class="multinomial").fit(train_x, train_y)
lr.score(test_x, test_y)

2) com PCA

In [ ]:
lr = lm.LogisticRegression(n_jobs = -1, solver="saga", multi_class="multinomial").fit(train_x, train_y)
lr.score(test_x_pca, test_y)

C. Rede neural de uma camada 

1) Sem PCA

In [ ]:
mlp = nn.MLPClassifier(hidden_layer_sizes=(2700, ), activation='logistic', solver='lbfgs',
                         learning_rate_init=0.001).fit(train_x, train_y)
print(mlp.score(test_x, test_y))

2) com PCA

In [ ]:
mlp = nn.MLPClassifier(hidden_layer_sizes=(5000, ), activation='logistic', solver='lbfgs',
                         learning_rate_init=0.001).fit(train_x_pca , train_y)
print(mlp.score(test_x_pca, test_y))

D. Rede neural de duas camada 

1) Sem PCA

In [ ]:
mlp = nn.MLPClassifier(hidden_layer_sizes=(x, y), activation='relu', solver='adam', 
                         learning_rate_init=0.001).fit(train_x, train_y)
print(mlp.score(test_x, test_y))

2) com PCA

In [ ]:
mlp = nn.MLPClassifier(hidden_layer_sizes=(x, y), activation='relu', solver='adam', 
                         learning_rate_init=0.001).fit(train_x_pca, train_y)
print(mlp.score(test_x_pca, test_y))